# ML Feature Engineering for Cost Estimation - Complete Demo

This notebook demonstrates the complete ML feature engineering pipeline for CPT cost estimation, from raw data to trained models with explainability.

## Table of Contents
1. [Setup and Data Preparation](#setup)
2. [Feature Engineering Pipeline](#feature-engineering)
3. [Data Preprocessing](#preprocessing)
4. [Model Training and Evaluation](#model-training)
5. [Model Explainability](#explainability)
6. [Integration with Cost Estimator](#integration)
7. [Performance Analysis](#performance)
8. [Production Deployment](#deployment)

## 1. Setup and Data Preparation {#setup}

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Import ML feature engineering components
import sys
sys.path.append('../')

from sdb.ml_feature_engineering import ComprehensiveFeatureEngineering, FeatureConfig
from sdb.ml_preprocessing import MLPreprocessingPipeline, create_default_preprocessing_pipeline
from sdb.ml_cost_estimator import MLCostEstimator, MLModelConfig, create_ml_enhanced_cost_estimator
from sdb.ml_validation import run_comprehensive_validation, FeatureQualityAnalyzer
from sdb.ml_explainability import ModelExplainer, create_feature_importance_dashboard
from sdb.cost_estimator import CptCost

print("ML Feature Engineering System - Complete Demo")
print("=" * 50)
print(f"Demo started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

### Create Realistic Test Dataset

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Define realistic CPT test data
base_tests = [
    # Laboratory Tests
    ('complete blood count', '85027', 9.0, 'laboratory'),
    ('comprehensive metabolic panel', '80053', 14.0, 'laboratory'),
    ('basic metabolic panel', '80048', 13.0, 'laboratory'),
    ('lipid panel', '80061', 15.0, 'laboratory'),
    ('thyroid stimulating hormone', '84443', 20.0, 'laboratory'),
    ('hemoglobin a1c', '83036', 13.0, 'laboratory'),
    ('vitamin d 25-hydroxy', '82306', 35.0, 'laboratory'),
    ('prostate specific antigen', '84153', 25.0, 'laboratory'),
    ('urinalysis complete', '81001', 5.0, 'laboratory'),
    ('blood culture aerobic', '87040', 20.0, 'laboratory'),
    
    # Imaging Tests
    ('chest x-ray 2 views', '71046', 30.0, 'imaging'),
    ('ct head without contrast', '70450', 100.0, 'imaging'),
    ('mri brain with contrast', '70553', 400.0, 'imaging'),
    ('mri lumbar spine without contrast', '72148', 350.0, 'imaging'),
    ('ultrasound abdomen complete', '76700', 150.0, 'imaging'),
    ('mammography bilateral', '77067', 120.0, 'imaging'),
    ('ct chest with contrast', '71260', 200.0, 'imaging'),
    ('x-ray knee 2 views', '73060', 35.0, 'imaging'),
    
    # Cardiology Tests  
    ('electrocardiogram 12 lead', '93000', 10.0, 'cardiology'),
    ('echocardiogram complete', '93307', 180.0, 'cardiology'),
    ('stress test treadmill', '93017', 250.0, 'cardiology'),
    ('holter monitor 24 hour', '93224', 200.0, 'cardiology'),
    ('cardiac catheterization', '93458', 1200.0, 'cardiology'),
    
    # Procedures
    ('colonoscopy screening', '45378', 800.0, 'procedure'),
    ('upper endoscopy diagnostic', '43235', 600.0, 'procedure'),
    ('skin biopsy single lesion', '11100', 150.0, 'procedure'),
    ('joint injection knee', '20610', 75.0, 'procedure')
]

# Generate training dataset with variations
training_data = []
test_counts = {}  # Track how many of each test we generate

for test_name, cpt_code, base_price, category in base_tests:
    # Generate 30-80 variations per test (realistic sample sizes)
    n_samples = np.random.randint(30, 81)
    test_counts[test_name] = n_samples
    
    for i in range(n_samples):
        # Add realistic price variation (10-25% standard deviation)
        price_std = base_price * np.random.uniform(0.10, 0.25)
        price = max(1.0, np.random.normal(base_price, price_std))
        
        # Add some seasonal/regional variation
        if np.random.random() < 0.1:  # 10% chance of outlier
            price *= np.random.uniform(0.7, 1.4)
            
        training_data.append({
            'test_name': test_name,
            'cpt_code': cpt_code,
            'price': round(price, 2),
            'category': category
        })

# Convert to DataFrame
df_train = pd.DataFrame(training_data)

# Display dataset summary
print(f"Generated Training Dataset:")
print(f"  Total samples: {len(df_train):,}")
print(f"  Unique tests: {df_train['test_name'].nunique()}")
print(f"  Price range: ${df_train['price'].min():.2f} - ${df_train['price'].max():.2f}")
print(f"  Average price: ${df_train['price'].mean():.2f}")

# Show category distribution
print(f"\nCategory Distribution:")
category_dist = df_train['category'].value_counts()
for category, count in category_dist.items():
    print(f"  {category}: {count} samples ({count/len(df_train)*100:.1f}%)")

# Display sample data
print(f"\nSample Data:")
df_train.head(10)

### Visualize Dataset Characteristics

In [ ]:
# Create visualization of dataset characteristics
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Training Dataset Characteristics', fontsize=16)

# Price distribution
axes[0, 0].hist(df_train['price'], bins=30, alpha=0.7, edgecolor='black')
axes[0, 0].set_xlabel('Price ($)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Price Distribution')
axes[0, 0].axvline(df_train['price'].mean(), color='red', linestyle='--', label=f'Mean: ${df_train["price"].mean():.2f}')
axes[0, 0].legend()

# Price by category
df_train.boxplot(column='price', by='category', ax=axes[0, 1])
axes[0, 1].set_title('Price Distribution by Category')
axes[0, 1].set_xlabel('Category')
axes[0, 1].set_ylabel('Price ($)')
plt.setp(axes[0, 1].xaxis.get_majorticklabels(), rotation=45)

# Category distribution
category_counts = df_train['category'].value_counts()
axes[1, 0].pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%')
axes[1, 0].set_title('Test Category Distribution')

# Log price distribution (for better visualization of range)
axes[1, 1].hist(np.log1p(df_train['price']), bins=30, alpha=0.7, edgecolor='black')
axes[1, 1].set_xlabel('Log(Price + 1)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Log Price Distribution')

plt.tight_layout()
plt.show()

# Statistical summary
print("\nStatistical Summary:")
print(df_train.groupby('category')['price'].agg(['count', 'mean', 'std', 'min', 'max']).round(2))

## 2. Feature Engineering Pipeline {#feature-engineering}

In [ ]:
# Configure feature engineering pipeline
feature_config = FeatureConfig(
    include_text_features=True,
    include_hierarchical_features=True,
    include_interaction_features=True,
    include_temporal_features=False,
    max_tfidf_features=50,
    min_word_freq=2,
    ngram_range=(1, 2),
    handle_missing_values=True,
    remove_outliers=True,
    outlier_method="iqr",
    enable_feature_selection=False
)

print("Feature Engineering Configuration:")
print(f"  Text features: {feature_config.include_text_features}")
print(f"  Hierarchical features: {feature_config.include_hierarchical_features}")
print(f"  Interaction features: {feature_config.include_interaction_features}")
print(f"  Max TF-IDF features: {feature_config.max_tfidf_features}")
print(f"  N-gram range: {feature_config.ngram_range}")

### Extract Features from Sample Data

In [ ]:
# Create feature engineering pipeline
feature_engineer = ComprehensiveFeatureEngineering(feature_config)

# Prepare input data (required columns for feature engineering)
X_input = df_train[['test_name', 'cpt_code', 'price']].copy()
y_target = df_train['price'].copy()

print(f"Input data shape: {X_input.shape}")
print(f"Sample input data:")
print(X_input.head())

# Fit and transform features
print(f"\nExtracting features...")
start_time = datetime.now()

X_features = feature_engineer.fit_transform(X_input, y_target)

processing_time = (datetime.now() - start_time).total_seconds()

print(f"Feature extraction complete!")
print(f"  Processing time: {processing_time:.2f} seconds")
print(f"  Output shape: {X_features.shape}")
print(f"  Features generated: {X_features.shape[1]}")
print(f"  Features per second: {X_features.shape[1]/processing_time:.1f}")

### Analyze Generated Features

In [ ]:
# Get feature names and metadata
feature_names = feature_engineer.get_feature_names()
feature_metadata = feature_engineer.get_feature_metadata()

print(f"Feature Analysis:")
print(f"  Total features: {len(feature_names)}")

# Categorize features by type
feature_types = {}
for name, metadata in feature_metadata.items():
    ftype = metadata['type']
    if ftype not in feature_types:
        feature_types[ftype] = []
    feature_types[ftype].append(name)

print(f"\nFeature Types:")
for ftype, names in feature_types.items():
    print(f"  {ftype}: {len(names)} features")

# Show sample features from each category
print(f"\nSample Features by Category:")
for ftype, names in feature_types.items():
    sample_features = names[:5]  # Show first 5 features of each type
    print(f"  {ftype}:")
    for fname in sample_features:
        description = feature_metadata.get(fname, {}).get('description', 'No description')
        print(f"    - {fname}: {description}")
    if len(names) > 5:
        print(f"    ... and {len(names) - 5} more")
    print()

### Feature Quality Analysis

In [ ]:
# Convert features to DataFrame for analysis
feature_df = pd.DataFrame(X_features, columns=[f"feature_{i}" for i in range(X_features.shape[1])])

# Analyze feature quality
quality_analyzer = FeatureQualityAnalyzer(
    target_correlation_threshold=0.1,
    outlier_threshold=3.0,
    missing_threshold=0.05
)

print("Analyzing feature quality...")
quality_metrics = quality_analyzer.analyze_feature_quality(feature_df, y_target)
quality_report = quality_analyzer.generate_quality_report(quality_metrics)

# Display quality summary
summary = quality_report['summary']
print(f"\nFeature Quality Summary:")
print(f"  Total features analyzed: {summary['total_features']}")
print(f"  Average quality score: {summary['avg_quality_score']:.3f}")
print(f"  High quality features (>0.7): {summary['high_quality_features']}")
print(f"  Low quality features (<0.3): {summary['low_quality_features']}")
print(f"  Constant features: {summary['constant_features']}")
print(f"  Average target correlation: {summary['avg_target_correlation']:.3f}")

# Show problematic features
issues = quality_report['feature_issues']
print(f"\nFeature Issues:")
for issue_type, feature_list in issues.items():
    if feature_list:
        print(f"  {issue_type}: {len(feature_list)} features")
        if len(feature_list) <= 3:
            print(f"    {', '.join(feature_list)}")
        else:
            print(f"    {', '.join(feature_list[:3])}... and {len(feature_list)-3} more")

### Visualize Feature Distributions

In [ ]:
# Create feature quality visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Feature Quality Analysis', fontsize=16)

# Quality score distribution
quality_scores = [m.quality_score for m in quality_metrics]
axes[0, 0].hist(quality_scores, bins=20, alpha=0.7, edgecolor='black')
axes[0, 0].set_xlabel('Quality Score')
axes[0, 0].set_ylabel('Number of Features')
axes[0, 0].set_title('Feature Quality Score Distribution')
axes[0, 0].axvline(np.mean(quality_scores), color='red', linestyle='--', 
                   label=f'Mean: {np.mean(quality_scores):.3f}')
axes[0, 0].legend()

# Target correlation distribution
correlations = [abs(m.correlation_with_target) for m in quality_metrics if m.correlation_with_target > 0]
if correlations:
    axes[0, 1].hist(correlations, bins=20, alpha=0.7, edgecolor='black')
    axes[0, 1].set_xlabel('Absolute Target Correlation')
    axes[0, 1].set_ylabel('Number of Features')
    axes[0, 1].set_title('Target Correlation Distribution')
    axes[0, 1].axvline(np.mean(correlations), color='red', linestyle='--',
                       label=f'Mean: {np.mean(correlations):.3f}')
    axes[0, 1].legend()

# Missing value percentage
missing_pcts = [m.missing_percentage for m in quality_metrics]
axes[1, 0].hist(missing_pcts, bins=20, alpha=0.7, edgecolor='black')
axes[1, 0].set_xlabel('Missing Value Percentage')
axes[1, 0].set_ylabel('Number of Features')
axes[1, 0].set_title('Missing Values Distribution')

# Quality vs Correlation scatter
if correlations:
    corr_dict = {m.feature_name: abs(m.correlation_with_target) for m in quality_metrics if m.correlation_with_target > 0}
    quality_dict = {m.feature_name: m.quality_score for m in quality_metrics}
    
    # Match features that have both quality and correlation
    common_features = set(corr_dict.keys()) & set(quality_dict.keys())
    x_vals = [quality_dict[f] for f in common_features]
    y_vals = [corr_dict[f] for f in common_features]
    
    axes[1, 1].scatter(x_vals, y_vals, alpha=0.6)
    axes[1, 1].set_xlabel('Quality Score')
    axes[1, 1].set_ylabel('Absolute Target Correlation')
    axes[1, 1].set_title('Quality vs Target Correlation')
    
    # Add trend line
    if len(x_vals) > 1:
        z = np.polyfit(x_vals, y_vals, 1)
        p = np.poly1d(z)
        axes[1, 1].plot(sorted(x_vals), p(sorted(x_vals)), "r--", alpha=0.8)

plt.tight_layout()
plt.show()

## 3. Data Preprocessing {#preprocessing}

In [ ]:
# Create comprehensive preprocessing pipeline
print("Creating ML preprocessing pipeline...")

preprocessing_pipeline = create_default_preprocessing_pipeline()

# Fit and transform the data
print("Fitting preprocessing pipeline...")
start_time = datetime.now()

X_processed = preprocessing_pipeline.fit_transform(X_input, y_target)

preprocessing_time = (datetime.now() - start_time).total_seconds()

print(f"Preprocessing complete!")
print(f"  Processing time: {preprocessing_time:.2f} seconds")
print(f"  Input shape: {X_input.shape}")
print(f"  Output shape: {X_processed.shape}")
print(f"  Features generated: {X_processed.shape[1]}")

# Get preprocessing statistics
prep_stats = preprocessing_pipeline.get_preprocessing_stats()
print(f"\nPreprocessing Statistics:")
print(f"  Original samples: {prep_stats['original_samples']}")
print(f"  Final samples: {prep_stats['final_samples']}")
print(f"  Sample retention rate: {prep_stats['final_samples']/prep_stats['original_samples']*100:.1f}%")
print(f"  Original features: {prep_stats['original_features']}")
print(f"  Final features: {prep_stats['final_features']}")

# Validation stats
if 'validation_stats' in prep_stats:
    val_stats = prep_stats['validation_stats']
    print(f"\nData Validation:")
    print(f"  Rows removed: {val_stats.get('rows_removed', 0)}")
    print(f"  Removal percentage: {val_stats.get('removal_percentage', 0):.1f}%")
    print(f"  Unique CPT codes: {val_stats.get('unique_cpt_codes', 0)}")
    print(f"  Price range: ${val_stats.get('price_range', [0, 0])[0]:.2f} - ${val_stats.get('price_range', [0, 0])[1]:.2f}")

### Validate Preprocessing Pipeline

In [ ]:
# Run comprehensive validation
print("Running comprehensive pipeline validation...")

validation_results = run_comprehensive_validation(
    X_input, 
    y_target
)

# Display validation summary
summary = validation_results.get('summary', {})
print(f"\nValidation Results:")
print(f"  Overall success: {summary.get('overall_success', False)}")
print(f"  Tests passed: {summary.get('tests_passed', 0)}/{summary.get('total_tests', 0)}")
print(f"  Success rate: {summary.get('success_rate', 0):.1f}%")

# Show individual test results
pipeline_tests = validation_results.get('pipeline_tests', [])
print(f"\nIndividual Test Results:")
for test in pipeline_tests:
    status = "✅ PASS" if test['success'] else "❌ FAIL"
    exec_time = test.get('execution_time', 0)
    print(f"  {status} {test['test_name']}: {exec_time:.3f}s")
    
    if not test['success'] and test.get('error_message'):
        print(f"    Error: {test['error_message']}")

# Check for major issues
major_issues = summary.get('major_issues', [])
if major_issues:
    print(f"\n⚠️  Major Issues Found:")
    for issue in major_issues:
        print(f"  - {issue}")
else:
    print(f"\n✅ No major issues detected!")

## 4. Model Training and Evaluation {#model-training}

In [ ]:
# Split data for training and testing
from sklearn.model_selection import train_test_split

# Use processed features
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y_target, 
    test_size=0.2, 
    random_state=42,
    stratify=df_train['category']  # Ensure balanced split across categories
)

print(f"Data Split:")
print(f"  Training set: {X_train.shape[0]} samples")
print(f"  Test set: {X_test.shape[0]} samples")
print(f"  Features: {X_train.shape[1]}")
print(f"  Training target range: ${y_train.min():.2f} - ${y_train.max():.2f}")
print(f"  Test target range: ${y_test.min():.2f} - ${y_test.max():.2f}")

### Train Multiple Models

In [ ]:
# Train multiple models for comparison
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import time

# Define models to test
models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0)
}

# Train and evaluate each model
model_results = {}

print("Training and evaluating models...")
print("=" * 50)

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Time the training
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    # Make predictions
    start_time = time.time()
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    prediction_time = time.time() - start_time
    
    # Calculate metrics
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    
    # Calculate MAPE (Mean Absolute Percentage Error)
    test_mape = np.mean(np.abs((y_test - y_pred_test) / np.maximum(y_test, 1e-8))) * 100
    
    # Store results
    model_results[name] = {
        'model': model,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'test_mae': test_mae,
        'test_rmse': test_rmse,
        'test_mape': test_mape,
        'training_time': training_time,
        'prediction_time': prediction_time,
        'predictions': y_pred_test
    }
    
    # Print results
    print(f"  R² Score (train): {train_r2:.3f}")
    print(f"  R² Score (test): {test_r2:.3f}")
    print(f"  MAE (test): ${test_mae:.2f}")
    print(f"  RMSE (test): ${test_rmse:.2f}")
    print(f"  MAPE (test): {test_mape:.1f}%")
    print(f"  Training time: {training_time:.3f}s")
    print(f"  Prediction time: {prediction_time:.3f}s")
    
    # Performance grade
    if test_r2 >= 0.9:
        grade = "Excellent"
    elif test_r2 >= 0.8:
        grade = "Very Good"
    elif test_r2 >= 0.7:
        grade = "Good"
    elif test_r2 >= 0.6:
        grade = "Fair"
    else:
        grade = "Poor"
    
    print(f"  Performance Grade: {grade}")

print("\n" + "=" * 50)
print("Model training complete!")

### Model Comparison Visualization

In [ ]:
# Create model comparison visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Model Performance Comparison', fontsize=16)

# Performance metrics comparison
model_names = list(model_results.keys())
r2_scores = [model_results[name]['test_r2'] for name in model_names]
mae_scores = [model_results[name]['test_mae'] for name in model_names]
mape_scores = [model_results[name]['test_mape'] for name in model_names]
training_times = [model_results[name]['training_time'] for name in model_names]

# R² Score comparison
bars = axes[0, 0].bar(model_names, r2_scores)
axes[0, 0].set_ylabel('R² Score')
axes[0, 0].set_title('Model R² Score Comparison')
axes[0, 0].set_ylim(0, 1)
plt.setp(axes[0, 0].xaxis.get_majorticklabels(), rotation=45)

# Color bars based on performance
for bar, score in zip(bars, r2_scores):
    if score >= 0.8:
        bar.set_color('green')
    elif score >= 0.6:
        bar.set_color('orange')
    else:
        bar.set_color('red')

# Add value labels
for bar, score in zip(bars, r2_scores):
    height = bar.get_height()
    axes[0, 0].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{score:.3f}', ha='center', va='bottom')

# MAE comparison
axes[0, 1].bar(model_names, mae_scores)
axes[0, 1].set_ylabel('Mean Absolute Error ($)')
axes[0, 1].set_title('Model MAE Comparison')
plt.setp(axes[0, 1].xaxis.get_majorticklabels(), rotation=45)

# Training time comparison
axes[1, 0].bar(model_names, training_times)
axes[1, 0].set_ylabel('Training Time (seconds)')
axes[1, 0].set_title('Model Training Time Comparison')
plt.setp(axes[1, 0].xaxis.get_majorticklabels(), rotation=45)

# Prediction accuracy scatter plot (best model)
best_model_name = max(model_results.keys(), key=lambda x: model_results[x]['test_r2'])
best_predictions = model_results[best_model_name]['predictions']

axes[1, 1].scatter(y_test, best_predictions, alpha=0.6)
axes[1, 1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1, 1].set_xlabel('Actual Price ($)')
axes[1, 1].set_ylabel('Predicted Price ($)')
axes[1, 1].set_title(f'Prediction Accuracy - {best_model_name}')

plt.tight_layout()
plt.show()

print(f"\nBest performing model: {best_model_name}")
print(f"R² Score: {model_results[best_model_name]['test_r2']:.3f}")
print(f"MAE: ${model_results[best_model_name]['test_mae']:.2f}")

## 5. Model Explainability {#explainability}

In [ ]:
# Use the best performing model for explainability analysis
best_model = model_results[best_model_name]['model']

# Create feature names (simplified for demo)
feature_names = [f"feature_{i}" for i in range(X_train.shape[1])]

# Create model explainer
print(f"Creating model explainer for {best_model_name}...")
explainer = ModelExplainer(best_model, feature_names)

# Generate comprehensive model explanation
print("Generating model explanation...")
explanation = explainer.explain_model(
    X_train, y_train, 
    importance_methods=['builtin', 'permutation']
)

print(f"\nModel Explanation Summary:")
print(f"  Model: {explanation.model_name}")
print(f"  Features analyzed: {len(explanation.feature_importances)}")

# Performance summary
performance = explanation.model_performance
print(f"\nModel Performance:")
print(f"  R² Score: {performance['r2']:.3f}")
print(f"  MAE: ${performance['mae']:.2f}")
print(f"  RMSE: ${performance['rmse']:.2f}")
print(f"  Mean Prediction: ${performance['mean_prediction']:.2f}")

### Feature Importance Analysis

In [ ]:
# Analyze feature importance
builtin_features = [f for f in explanation.feature_importances if f.importance_type == 'builtin']
perm_features = [f for f in explanation.feature_importances if f.importance_type == 'permutation']

print(f"Feature Importance Analysis:")
print(f"  Built-in importance features: {len(builtin_features)}")
print(f"  Permutation importance features: {len(perm_features)}")

# Top features by built-in importance
if builtin_features:
    top_builtin = sorted(builtin_features, key=lambda x: x.importance_score, reverse=True)[:10]
    print(f"\nTop 10 Features (Built-in Importance):")
    for i, feature in enumerate(top_builtin, 1):
        print(f"  {i:2d}. {feature.feature_name}: {feature.importance_score:.4f}")

# Top features by permutation importance
if perm_features:
    top_perm = sorted(perm_features, key=lambda x: x.importance_score, reverse=True)[:10]
    print(f"\nTop 10 Features (Permutation Importance):")
    for i, feature in enumerate(top_perm, 1):
        ci_text = ""
        if feature.confidence_interval:
            ci_width = feature.confidence_interval[1] - feature.confidence_interval[0]
            ci_text = f" (±{ci_width/2:.4f})"
        stability_text = ""
        if feature.stability_score:
            stability_text = f" [stability: {feature.stability_score:.2f}]"
        print(f"  {i:2d}. {feature.feature_name}: {feature.importance_score:.4f}{ci_text}{stability_text}")

### Create Feature Importance Dashboard

In [ ]:
# Create comprehensive feature importance dashboard
dashboard_fig = create_feature_importance_dashboard(explanation)
plt.show()

# Display global explanation
print(f"\nGlobal Model Explanation:")
print("=" * 50)
print(explanation.global_explanation)

### Individual Prediction Explanations

In [ ]:
# Explain individual predictions for interesting samples
test_indices = [
    np.argmax(y_test),  # Most expensive test
    np.argmin(y_test),  # Least expensive test
    len(y_test) // 2    # Median test
]

sample_descriptions = ["Most Expensive", "Least Expensive", "Median Price"]

print(f"\nIndividual Prediction Explanations:")
print("=" * 50)

for i, idx in enumerate(test_indices):
    description = sample_descriptions[i]
    
    # Get prediction explanation
    pred_explanation = explainer.explain_prediction(
        X_test[idx], 
        sample_id=f"sample_{idx}",
        actual_value=y_test.iloc[idx]
    )
    
    print(f"\n{description} Sample (Index: {idx}):")
    print(f"  Predicted: ${pred_explanation.prediction:.2f}")
    print(f"  Actual: ${pred_explanation.actual_value:.2f}")
    
    error = abs(pred_explanation.prediction - pred_explanation.actual_value)
    error_pct = error / pred_explanation.actual_value * 100
    print(f"  Error: ${error:.2f} ({error_pct:.1f}%)")
    
    print(f"  Top Contributing Features:")
    for j, (feature, contribution) in enumerate(pred_explanation.top_contributing_features[:5], 1):
        direction = "↑" if contribution > 0 else "↓"
        print(f"    {j}. {feature}: {direction} ${abs(contribution):.2f}")
    
    if pred_explanation.explanation_text:
        print(f"  Explanation:")
        # Split explanation into lines for better formatting
        lines = pred_explanation.explanation_text.split('\n')
        for line in lines:
            if line.strip():
                print(f"    {line.strip()}")

## 6. Integration with Cost Estimator {#integration}

In [ ]:
# Create original cost table from our base data
cost_table = {}
for test_name, cpt_code, base_price, category in base_tests[:10]:  # Use subset for demo
    cost_table[test_name] = CptCost(
        cpt_code=cpt_code,
        price=base_price,
        category=category
    )

print(f"Created cost table with {len(cost_table)} entries")

# Create ML-enhanced cost estimator
ml_config = MLModelConfig(
    model_type="random_forest",
    model_params={"n_estimators": 100, "random_state": 42},
    performance_threshold=0.7,
    enable_model_selection=True
)

print(f"\nCreating ML-enhanced cost estimator...")
ml_estimator = create_ml_enhanced_cost_estimator(
    cost_table=cost_table,
    training_data=df_train,
    ml_config=ml_config
)

# Display model performance
performance = ml_estimator.get_model_performance()
if performance:
    print(f"\nML Model Performance:")
    print(f"  Model: {performance['model_name']}")
    print(f"  Performance Grade: {ml_estimator.model_performance.performance_grade}")
    print(f"  R² Score: {performance['r2_score']:.3f}")
    print(f"  MAE: ${performance['mean_absolute_error']:.2f}")
    print(f"  Training Samples: {performance['training_samples']:,}")
    print(f"  Features: {performance['feature_count']}")
    print(f"  Training Time: {performance['training_time']:.2f}s")

### Test Integration with Different Scenarios

In [ ]:
# Test different scenarios
test_scenarios = [
    # Known tests (should use lookup)
    ('complete blood count', 'Known test - should use lookup'),
    ('chest x-ray 2 views', 'Known test - should use lookup'),
    
    # Similar to known tests (should use ML with high confidence)
    ('complete blood count with differential', 'Similar to known test'),
    ('chest x-ray single view', 'Similar to known test'),
    ('basic metabolic panel comprehensive', 'Similar to known test'),
    
    # Completely unknown tests (should use ML or fallback)
    ('advanced cardiac biomarkers panel', 'Unknown test - cardiac'),
    ('molecular genetic testing brca1', 'Unknown test - genetic'),
    ('pet scan whole body with contrast', 'Unknown test - imaging'),
    ('specialized neurological function test', 'Unknown test - neuro')
]

print(f"Testing Cost Estimation Integration:")
print("=" * 60)
print(f"{'Test Name':<40} {'Price':<10} {'Category':<12} {'Source':<15}")
print("-" * 60)

for test_name, description in test_scenarios:
    try:
        # Get cost estimate
        cost, category = ml_estimator.estimate(test_name)
        
        # Determine source
        source = "Lookup"
        try:
            # Check if it's in the original cost table
            ml_estimator.lookup_cost(test_name)
        except KeyError:
            # Not in lookup table
            if ml_estimator.ml_model:
                try:
                    ml_pred, confidence = ml_estimator.predict_ml_cost(test_name)
                    if confidence >= ml_estimator.confidence_threshold:
                        source = f"ML ({confidence:.2f})"
                    else:
                        source = "ML (low conf)"
                except:
                    source = "Fallback"
            else:
                source = "Fallback"
        
        # Display result
        print(f"{test_name:<40} ${cost:<9.2f} {category:<12} {source:<15}")
        
    except Exception as e:
        print(f"{test_name:<40} {'ERROR':<10} {'N/A':<12} {str(e)[:15]:<15}")

print("-" * 60)
print("Source Legend: Lookup=Direct table lookup, ML=Machine Learning, Fallback=LLM or average")

### Get Prediction Explanations

In [ ]:
# Get explanations for some predictions
explanation_tests = [
    'advanced cardiac biomarkers panel',
    'pet scan whole body with contrast',
    'basic metabolic panel comprehensive'
]

print(f"\nPrediction Explanations:")
print("=" * 50)

for test_name in explanation_tests:
    explanation = ml_estimator.explain_prediction(test_name)
    
    if 'error' in explanation:
        print(f"\n{test_name}:")
        print(f"  Error: {explanation['error']}")
        continue
    
    print(f"\n{test_name}:")
    print(f"  Predicted Cost: ${explanation['prediction']:.2f}")
    
    if 'top_features' in explanation:
        print(f"  Top Contributing Features:")
        for i, (feature, contribution) in enumerate(explanation['top_features'][:3], 1):
            direction = "increases" if contribution > 0 else "decreases" 
            print(f"    {i}. {feature}: {direction} cost by ${abs(contribution):.2f}")
    
    if 'explanation_text' in explanation and explanation['explanation_text']:
        print(f"  Explanation: {explanation['explanation_text'].split('.')[0]}.")

## 7. Performance Analysis {#performance}

In [ ]:
# Comprehensive performance analysis
print(f"Comprehensive Performance Analysis")
print("=" * 50)

# Feature engineering performance
feature_processing_rate = len(df_train) / preprocessing_time
print(f"\nFeature Engineering Performance:")
print(f"  Total samples processed: {len(df_train):,}")
print(f"  Processing time: {preprocessing_time:.2f} seconds")
print(f"  Processing rate: {feature_processing_rate:.0f} samples/second")
print(f"  Features generated: {X_processed.shape[1]}")
print(f"  Feature generation rate: {X_processed.shape[1]/preprocessing_time:.1f} features/second")

# Model performance by category
print(f"\nModel Performance by Test Category:")
print(f"{'Category':<15} {'Samples':<8} {'R² Score':<10} {'MAE':<10} {'MAPE':<10}")
print("-" * 55)

# Split test data by category for analysis
test_df = df_train.iloc[X_test.index] if hasattr(X_test, 'index') else df_train[-len(y_test):]
best_predictions = model_results[best_model_name]['predictions']

for category in df_train['category'].unique():
    # Get indices for this category in test set
    if hasattr(test_df, 'reset_index'):
        category_mask = test_df.reset_index()['category'] == category
    else:
        category_mask = test_df['category'] == category
    
    if category_mask.sum() > 0:
        y_cat_true = y_test[category_mask]
        y_cat_pred = best_predictions[category_mask]
        
        if len(y_cat_true) > 1:
            cat_r2 = r2_score(y_cat_true, y_cat_pred)
            cat_mae = mean_absolute_error(y_cat_true, y_cat_pred)
            cat_mape = np.mean(np.abs((y_cat_true - y_cat_pred) / np.maximum(y_cat_true, 1e-8))) * 100
            
            print(f"{category:<15} {len(y_cat_true):<8} {cat_r2:<10.3f} ${cat_mae:<9.2f} {cat_mape:<9.1f}%")

# Overall system performance
print(f"\nOverall System Performance:")
total_pipeline_time = preprocessing_time + model_results[best_model_name]['training_time']
print(f"  End-to-end training time: {total_pipeline_time:.2f} seconds")
print(f"  Prediction latency: {model_results[best_model_name]['prediction_time']/len(y_test)*1000:.1f} ms/sample")
print(f"  Memory footprint: ~{X_processed.nbytes / 1024 / 1024:.1f} MB (features only)")
print(f"  Model accuracy: {model_results[best_model_name]['test_r2']:.1%}")
print(f"  Model precision: ${model_results[best_model_name]['test_mae']:.2f} MAE")

### Performance Benchmarking

In [ ]:
# Benchmark different data sizes
print(f"\nPerformance Scaling Analysis:")
print(f"{'Data Size':<12} {'Processing Time':<16} {'Features':<10} {'Rate (samples/s)':<15}")
print("-" * 55)

# Test with different sample sizes
test_sizes = [100, 500, 1000, len(df_train)]

for size in test_sizes:
    if size <= len(df_train):
        # Sample data
        sample_data = df_train.sample(n=size, random_state=42)
        sample_input = sample_data[['test_name', 'cpt_code', 'price']]
        
        # Time the processing
        start_time = time.time()
        
        # Create fresh pipeline for fair comparison
        sample_pipeline = create_default_preprocessing_pipeline()
        sample_features = sample_pipeline.fit_transform(sample_input)
        
        process_time = time.time() - start_time
        rate = size / process_time
        
        print(f"{size:<12,} {process_time:<16.3f} {sample_features.shape[1]:<10} {rate:<15.0f}")

# Memory usage estimation
print(f"\nMemory Usage Estimation:")
feature_memory = X_processed.nbytes / 1024 / 1024
print(f"  Feature matrix: {feature_memory:.1f} MB")
print(f"  Per sample: {feature_memory / len(X_processed) * 1024:.1f} KB")
print(f"  Estimated for 100K samples: {feature_memory / len(X_processed) * 100000:.0f} MB")

## 8. Production Deployment Considerations {#deployment}

In [ ]:
# Save the trained model for production deployment
model_save_path = "../models/ml_cost_estimator_demo.pkl"

print(f"Production Deployment Checklist:")
print("=" * 40)

# Save model
try:
    import os
    os.makedirs("../models", exist_ok=True)
    ml_estimator.save_ml_model(model_save_path)
    print(f"✅ Model saved to {model_save_path}")
except Exception as e:
    print(f"❌ Model save failed: {e}")

# Model validation checklist
print(f"\n📋 Model Validation Checklist:")
validation_checks = [
    ("Model R² Score > 0.7", model_results[best_model_name]['test_r2'] > 0.7),
    ("Model MAE < $50", model_results[best_model_name]['test_mae'] < 50),
    ("Training completed successfully", ml_estimator.model_performance is not None),
    ("Feature engineering working", X_processed.shape[1] > 0),
    ("No major validation failures", len(summary.get('major_issues', [])) == 0),
    ("Prediction explanations available", ml_estimator.model_explainer is not None)
]

for check, passed in validation_checks:
    status = "✅" if passed else "❌"
    print(f"  {status} {check}")

# Performance requirements
print(f"\n⚡ Performance Requirements:")
perf_requirements = [
    ("Prediction latency < 100ms", model_results[best_model_name]['prediction_time']/len(y_test) < 0.1),
    ("Training time < 60s", model_results[best_model_name]['training_time'] < 60),
    ("Memory usage reasonable", feature_memory < 500),  # Less than 500MB
    ("Feature processing < 10s", preprocessing_time < 10)
]

for requirement, met in perf_requirements:
    status = "✅" if met else "⚠️"
    print(f"  {status} {requirement}")

# Deployment recommendations
print(f"\n🚀 Deployment Recommendations:")
print(f"  • Enable feature caching for frequently requested tests")
print(f"  • Implement prediction confidence thresholds (current: {ml_estimator.confidence_threshold})")
print(f"  • Set up model performance monitoring")
print(f"  • Plan for regular model retraining (monthly/quarterly)")
print(f"  • Implement A/B testing for model updates")
print(f"  • Set up data quality monitoring for input features")
print(f"  • Configure fallback mechanisms for ML failures")

# Production integration example
print(f"\n🔧 Production Integration Example:")
print(f"""```python
# Load trained model in production
from sdb.ml_cost_estimator import MLCostEstimator

# Load model
estimator = MLCostEstimator(cost_table)
estimator.load_ml_model('{model_save_path}')

# Make predictions with confidence checking
cost, category = estimator.estimate('new test name')
explanation = estimator.explain_prediction('new test name')

# Monitor performance
performance = estimator.get_model_performance()
if performance['r2_score'] < 0.7:
    # Trigger retraining alert
    pass
```""")

## Summary and Conclusions

In [ ]:
# Final summary
print(f"\n" + "=" * 60)
print(f"ML FEATURE ENGINEERING SYSTEM - DEMO SUMMARY")
print(f"=" * 60)

print(f"\n📊 Dataset Summary:")
print(f"  • Training samples: {len(df_train):,}")
print(f"  • Unique tests: {df_train['test_name'].nunique()}")
print(f"  • Price range: ${df_train['price'].min():.2f} - ${df_train['price'].max():.2f}")
print(f"  • Test categories: {df_train['category'].nunique()}")

print(f"\n🔧 Feature Engineering Results:")
print(f"  • Features generated: {X_processed.shape[1]}")
print(f"  • Processing time: {preprocessing_time:.2f} seconds")
print(f"  • Processing rate: {len(df_train)/preprocessing_time:.0f} samples/second")
print(f"  • Feature quality: {summary.get('avg_quality_score', 0):.3f} average score")

print(f"\n🤖 Model Performance:")
print(f"  • Best model: {best_model_name}")
print(f"  • R² Score: {model_results[best_model_name]['test_r2']:.3f}")
print(f"  • Mean Absolute Error: ${model_results[best_model_name]['test_mae']:.2f}")
print(f"  • Mean Absolute Percentage Error: {model_results[best_model_name]['test_mape']:.1f}%")
print(f"  • Performance Grade: {MLCostEstimator(cost_table).model_performance.performance_grade if MLCostEstimator(cost_table).model_performance else 'N/A'}")

print(f"\n🎯 System Capabilities:")
print(f"  ✅ Comprehensive feature extraction from CPT codes and test names")
print(f"  ✅ Advanced data preprocessing with quality validation")
print(f"  ✅ Multiple ML model support with automated selection")
print(f"  ✅ Feature importance analysis and model explainability")
print(f"  ✅ Seamless integration with existing cost estimator")
print(f"  ✅ Production-ready deployment with monitoring")

print(f"\n🚀 Ready for Production:")
all_checks_passed = all([check[1] for check in validation_checks])
if all_checks_passed:
    print(f"  ✅ All validation checks passed")
    print(f"  ✅ Performance requirements met")
    print(f"  ✅ Model saved and ready for deployment")
    print(f"  ✅ System is production-ready!")
else:
    failed_checks = [check[0] for check in validation_checks if not check[1]]
    print(f"  ⚠️  Some validation checks failed: {', '.join(failed_checks)}")
    print(f"  📝 Review and address issues before production deployment")

print(f"\n📈 Next Steps:")
print(f"  1. Deploy model to production environment")
print(f"  2. Implement monitoring and alerting")
print(f"  3. Set up automated retraining pipeline")
print(f"  4. Conduct A/B testing with current system")
print(f"  5. Monitor performance and gather feedback")

print(f"\n" + "=" * 60)
print(f"Demo completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"=" * 60)

## Additional Resources

For more information about the ML Feature Engineering System:

- **Documentation**: See `docs/ml_feature_engineering_guide.md` for comprehensive usage guide
- **API Reference**: Detailed API documentation in the guide
- **Performance Benchmarks**: Complete benchmarking results in the documentation
- **Troubleshooting**: Common issues and solutions in the guide
- **Best Practices**: Production deployment recommendations

### Key Components Used:

1. **`sdb.ml_feature_engineering`**: Core feature engineering pipeline
2. **`sdb.ml_preprocessing`**: Data preprocessing and validation
3. **`sdb.ml_cost_estimator`**: ML-enhanced cost estimation
4. **`sdb.ml_validation`**: Comprehensive testing framework
5. **`sdb.ml_explainability`**: Model interpretation and explanations
6. **`sdb.ml_feature_store`**: Feature storage and caching system

The system is now ready for production deployment with robust feature engineering, model training, and explainability capabilities.